# Entropy and Persistent Homology

## &copy;  [Omkar Mehta](omehta2@illinois.edu) ##
### Industrial and Enterprise Systems Engineering, The Grainger College of Engineering,  UIUC ###

<hr style="border:2px solid blue"> </hr>

# [Reference](https://towardsdatascience.com/how-to-pull-data-from-an-api-using-python-requests-edcc8d6441b1)

# Part 1: Download Data from INaturalist website using API
From API documentation,we get the following information:

`Please note that we throttle API usage to a max of 100 requests per minute, though we ask that you try to keep it to 60 requests per minute or lower, and to keep under 10,000 requests per day. If we notice usage that has serious impact on our performance we may institute blocks without notification.`

`per_page
Allowed values: 1 to 200`

Locations data collected from:
* place_id: 1563
* place_id: 49906

In [2]:
# importing required libraries
import requests
import math
import time
import pandas as pd
from datetime import datetime
import os

In [3]:
# Data for storing all the pulled data
data = {
    'time_observed_at': list(),
    'species_guess': list(),
    'genus_name': list(),
    'rank': list(),
    'wikipedia_url': list(),
    'iconic_taxon_name': list(),
    'preferred_common_name': list(),
    'uri': list(),
    'longitude': list(),
    'latitude': list(),
    'place_guess': list()
}

## Part 1.1. Function for getting the dictionary of pages and page_numbers

For each page, there is a different per_page limit. This was not mentioned in the api documentation of the inaturalist website.

In [4]:
# pages = {}
# place_id = 1563
# page_number = 1
# per_page = 200 #max_limit
# i = 1
# sleep_time = 60
# while True:
#   while per_page != 0:
#     #go through each page and per_page, get the request's response, if it is 200, append page_number:per_page to the dictionary.
#     #if for any per_page, we get status_code != 200, we decrement the per_page by 1, and check response. 
    
#     response = requests.get("https://api.inaturalist.org/v1/observations?place_id={}&page={}&per_page={}".format(place_id, page_number, per_page))
#     if i%50 == 0: #We have a limit of 60 pages per minute, for pulling data from API
#         time.sleep(sleep_time)
#     if response.status_code == 200:
#       pages[page_number] = per_page
#       page_number +=1
#     elif response.status_code != 200:
#       per_page -= 1
#     i += 1
#   break 

def findPerPage(place_id, page_number, per_page, how_many_pages):
    pages = {}
    i = 1
    sleep_time = 60
    while True:
        while per_page != 0:
            #go through each page and per_page, get the request's response, 
            # if it is 200, append page_number:per_page to the dictionary.
            # if for any per_page, we get status_code != 200, we decrement the per_page by 1, and check response. 
            response = requests.get("https://api.inaturalist.org/v1/observations?place_id={}&page={}&per_page={}".format(place_id, page_number, per_page))
            if i%50 == 0: #We have a limit of 60 pages per minute, for pulling data from API
                time.sleep(sleep_time)
            if response.status_code == 200:
                pages[page_number] = per_page
                page_number +=1
            elif response.status_code != 200:
                per_page -= 1
            i += 1

            # if how_many_pages >= 200:
            #     return pages
        break
    return pages

def download_csv_pages(pages):
    pages_Series = pd.Series(pages)
    #pages_Series.to_csv('pages.csv')
    pages_df = pd.DataFrame(pages_Series, columns=['page_number', 'per_page'])
    pages_df.to_csv('pages.csv')

def download_pickle_pages(pages):
    pages_Series = pd.Series(pages)
    #pages_Series.to_csv('pages.csv')
    pages_df = pd.DataFrame(pages_Series, columns=['page_number', 'per_page'])
    pages_df.to_pickle('pages.pkl')

In [5]:
# pages_Series = pd.Series(pages)
# pages_Series.to_csv('pages.csv')
# pages_df = pd.read_csv('pages.csv', names=['page_number', 'per_page'])
# pages_df.info()
# pages_df.to_csv('pages.csv')
# pages_df.to_pickle('pages.pkl')

## Part 1.2. Pull data

In [6]:
# Function for pulling the data
def pull_data(data, place_id, sleep_time, page_number, per_page):
    '''
    place_id: each location has a place_id, which we can get from the website
    sleep_time: time in seconds we want to sleep
    page_number: depends on the number of observations. total_observations/per_page
    per_page: #observations per page
    '''
    if page_number%40 == 0:
        time.sleep(sleep_time)
    else:
        response = requests.get("https://api.inaturalist.org/v1/observations?place_id={}&page={}&per_page={}".format(place_id, page_number, per_page))
        file_dict = response.json()
        if ('results' in file_dict):
            for j in range(per_page):
            
                if 'time_observed_at' in file_dict['results'][j]:
                    data['time_observed_at'].append(file_dict['results'][j]['time_observed_at'])
                else:
                    data['time_observed_at'].append(None)
                if 'species_guess' in file_dict['results'][j]:
                    data['species_guess'].append(file_dict['results'][j]['species_guess'])
                else:
                    data['species_guess'].append(None)

                if ('taxon' in file_dict['results'][j]) & (file_dict['results'][j]['taxon'] is not None) :
                    #print(j)
                    if 'name' in file_dict['results'][j]['taxon']:
                        data['genus_name'].append(file_dict['results'][j]['taxon']['name'])
                    else:
                        data['genus_name'].append(None)
                    if 'rank' in file_dict['results'][j]['taxon']:
                        data['rank'].append(file_dict['results'][j]['taxon']['rank'])
                    else:
                        data['rank'].append(None)
                    if 'wikipedia_url' in file_dict['results'][j]['taxon']:
                        data['wikipedia_url'].append(file_dict['results'][j]['taxon']['wikipedia_url'])
                    else:
                        data['wikipedia_url'].append(None)
                    if 'iconic_taxon_name' in file_dict['results'][j]['taxon']:
                        data['iconic_taxon_name'].append(file_dict['results'][j]['taxon']['iconic_taxon_name'])
                    else:
                        data['iconic_taxon_name'].append(None)
                    #print(j)
                    if 'preferred_common_name' in file_dict['results'][j]['taxon']:
                        data['preferred_common_name'].append(file_dict['results'][j]['taxon']['preferred_common_name'])
                    else:
                        data['preferred_common_name'].append(None)
                else:
                    data['genus_name'].append(None)
                    data['rank'].append(None)
                    data['wikipedia_url'].append(None)
                    data['iconic_taxon_name'].append(None)
                    data['preferred_common_name'].append(None)
                if 'uri' in file_dict['results'][j]:
                    data['uri'].append(file_dict['results'][j]['uri'])
                else:
                    data['uri'].append(None)
                if 'geojson' in file_dict['results'][j]:
                    data['longitude'].append(file_dict['results'][j]['geojson']['coordinates'][0])
                    data['latitude'].append(file_dict['results'][j]['geojson']['coordinates'][1])
                else:
                    data['longitude'].append(None)
                    data['latitude'].append(None)
                if 'place_guess' in file_dict['results'][j]:
                    data['place_guess'].append(file_dict['results'][j]['place_guess'])
                else:
                    data['place_guess'].append(None)
        else:
            print(page_number)

    

`pages.csv' contains the 
```python
{'page_number': 'per_page'}
```
dictionary. For each page, we have a maximum limit on the number of observations that one can pull.

In [7]:
pages = pd.read_csv('pages.csv')

In [9]:
# Uncomment for pulling data. Change place_id
# for i in range(1, len(pages)):

#     pull_data(data, 1563, 60, int(pages['page_number'][i]), int(pages['per_page'][i]))
# data_df = pd.DataFrame(data)
# data_df.info()

# place_id = 1563
# csv_file = 'data_' + str(place_id) + '_' + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + '.csv'
# pickle_file = 'data_' + str(place_id) + '_' + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + '.pkl'
# data_df.to_csv(csv_file)
# data_df.to_pickle(pickle_file)
# data_df.info()

def getData(pages_df, data, place_id):

    for i in range(1, len(pages)):
        pull_data(data, place_id, 60, int(pages_df['page_number'][i]), int(pages_df['per_page'][i]))
    data_df = pd.DataFrame(data)
    return data_df
def download_csv_data(data, place_id, ):

    csv_file = 'data_' + str(place_id) + '_' + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + '.csv'
    pickle_file = 'data_' + str(place_id) + '_' + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + '.pkl'
    data_df.to_csv(csv_file)
    data_df.to_pickle(pickle_file)


***Use pickle instead of csv***

In [10]:
data_df = pd.read_csv('data_1563_2021_06_24_12_57_28.csv')
# data_df.to_pickle('data_1563_2021_06_06_20_47_20.pickle')
# data_df = pd.read_pickle('data_1563_2021_06_06_20_47_20.pickle')
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45576 entries, 0 to 45575
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             45576 non-null  int64  
 1   time_observed_at       45240 non-null  object 
 2   species_guess          31406 non-null  object 
 3   genus_name             44627 non-null  object 
 4   rank                   44627 non-null  object 
 5   wikipedia_url          42470 non-null  object 
 6   iconic_taxon_name      44611 non-null  object 
 7   preferred_common_name  42654 non-null  object 
 8   uri                    45576 non-null  object 
 9   longitude              45576 non-null  float64
 10  latitude               45576 non-null  float64
 11  place_guess            45576 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 4.2+ MB


In [15]:
data_df.head()

,Unnamed: 0,time_observed_at,species_guess,genus_name,rank,wikipedia_url,iconic_taxon_name,preferred_common_name,uri,longitude,latitude,place_guess
0,0,2021-06-23T16:22:12-05:00,NaN,Storeria dekayi,species,http://en.wikipedia.org/wiki/Storeria_dekayi,Reptilia,Dekay's Brownsnake,https://www.inaturalist.org/observations/84313002,-88.367813,40.212863,"Mahomet, IL, US"
1,1,2021-06-24T10:29:25-05:00,NaN,Potentilla norvegica,species,http://en.wikipedia.org/wiki/Potentilla_norvegica,Plantae,rough cinquefoil,https://www.inaturalist.org/observations/84311385,-88.369087,40.213030,"N Lake of the Woods Rd, Mahomet, IL, US"
2,2,2021-06-23T14:19:35-05:00,NaN,Tettigoniinae,subfamily,http://en.wikipedia.org/wiki/Tettigoniinae,Insecta,Shieldback Katydids,https://www.inaturalist.org/observations/84311301,-88.368783,40.210567,"Mahomet, IL, US"
3,3,2021-06-24T09:45:31+00:00,NaN,Marpissa lineata,species,http://en.wikipedia.org/wiki/Marpissa_lineata,Arachnida,Four-lined Slender Jumping Spider,https://www.inaturalist.org/observations/84306238,-88.208083,40.112786,"W Main St, Urbana, IL, US"
4,4,2021-06-24T06:03:52-05:00,Eastern Box Turtle,Terrapene carolina carolina,subspecies,https://en.wikipedia.org/wiki/Eastern_box_turtle,Reptilia,Eastern Box Turtle,https://www.inaturalist.org/observations/84287773,-88.164075,40.030186,"Illinois, US"


# Part 2. Get taxonomy data from the rows

## Search for a term in wikipedia search and get its page

Uncomment for looking at results. I didn't find it useful.

In [2]:
# import wikipedia
# result = wikipedia.search("Monarda_fistulosa")
# print(result)
# # get the page: Neural network
# page = wikipedia.page(result[0])
# print(page)
# # get the title of the page
# title = page.title
# print(title)
# # get the categories of the page
# categories = page.categories
# print(categories)
# # get the whole wikipedia page text (content)
# content = page.content
# print(content)
# # get all the links in the page
# links = page.links
# print(links)
# # get the page references
# references = page.references
# print(references)
# # summary
# summary = page.summary
# print(summary)

## Part 2.1. Scraping data using beautiful soup


In [39]:
# For each row, if wikipedia_uri exists, we will use beautiful soup to extract the data 
# related to taxonomy
# import required modules
import requests
from bs4 import BeautifulSoup
# get URL
page = requests.get("http://en.wikipedia.org/wiki/Monarda_fistulosa")
  
# display status code
# print(page.status_code)
  
# display scrapped data
# print(page.content)

# scrape webpage
soup = BeautifulSoup(page.content, 'html.parser') #.get_text(strip=True) #This removes \xa0

# display scrapped data
# print(soup.prettify())

# list(soup.children)

# find all occurance of p in HTML
# includes HTML tags
# print(soup.find_all('p'))

In [40]:
data_df[data_df['rank'] == 'species' ]['wikipedia_url'][5]

'http://en.wikipedia.org/wiki/Aesculus_parviflora'

### Part 3.1.1. Get table from the id

In [65]:
# create object
# object = soup.find(id="mw-content-text")

# # find tags
# items = object.find_all(class_="infobox biota")
# result = items[0]
  
# # display tags
# print(result.prettify())

table = soup.find_all('table')

# table[0]

# for child in soup.find_all('table')[0].children:
#     for td in child:
#         print(td)

# list(soup.find_all('table')[0].tr.next_siblings)

# for sibling in soup.find_all('table')[0].tr.next_siblings:
#     for td in sibling:
#         print(td)

table = soup.find('table', attrs={'class':'infobox biota'}) #class="infobox biota"
table_rows = table.find_all('tr')

# table_rows

# len(table_rows)
data_taxonomy = {
    'Kingdom': list(),
    'Phylum': list(),
    'Class': list(),
    'Order': list(),
    'Suborder': list(),
    'Family': list(),
    'Genus': list(),
    'Species': list()
}
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.replace('\n', '').replace(':', '').replace(u'\xa0', ' ') for tr in td]
    data_taxonomy['Kingdom'].append(None)
    data_taxonomy['Phylum'].append(None)
    data_taxonomy['Class'].append(None)
    data_taxonomy['Order'].append(None)
    data_taxonomy['Suborder'].append(None)
    data_taxonomy['Family'].append(None)
    data_taxonomy['Genus'].append(None)
    data_taxonomy['Species'].append(None)

    if 'Kingdom' in row:
        data_taxonomy['Kingdom'] = row[1]
        l.append(row)
    if 'Phylum' in row:
        data_taxonomy['Phylum'] = row[1]
        l.append(row)   
    if 'Class' in row:
        data_taxonomy['Class'] = row[1]
        l.append(row)
    if 'Order' in row:
        data_taxonomy['Order'] = row[1]
        l.append(row)
    if 'Suborder' in row:
        data_taxonomy['Suborder'] = row[1]
        l.append(row)
    if 'Family' in row:
        data_taxonomy['Family'] = row[1]
        l.append(row)
    if 'Genus' in row:
        data_taxonomy['Genus'] = row[1]
        l.append(row)
    if 'Species' in row:
        data_taxonomy['Species'] = row[1]
        l.append(row)
    
    # elif 'Clade' in row:
    #     l.append(row)
l

# 'Kingdom' in l[4]

# table_rows = table_rows[4:13]

# l = []
# for tr in table_rows:
#     td = tr.find_all('td')
#     row = [tr.text for tr in td]
#     l.append(row)

# l

[['Kingdom', 'Plantae'],
 ['Order', 'Lamiales'],
 ['Family', 'Lamiaceae'],
 ['Genus', 'Monarda'],
 ['Species', 'M. fistulosa']]

In [66]:
data_taxonomy

{'Kingdom': ['Plantae'],
 'Phylum': [],
 'Class': [],
 'Order': ['Lamiales'],
 'Suborder': [],
 'Family': ['Lamiaceae'],
 'Genus': ['Monarda'],
 'Species': ['M. fistulosa']}

In [47]:
for item in l:
    print(item)

['Kingdom', 'Plantae']
['Clade', 'Tracheophytes']
['Clade', 'Angiosperms']
['Clade', 'Eudicots']
['Clade', 'Asterids']
['Order', 'Lamiales']
['Family', 'Lamiaceae']
['Genus', 'Monarda']
['Species', 'M. fistulosa']


In [48]:
taxonomy = {'taxonomy': list()}

taxonomy['taxonomy'].append({item[0]: item[1] for item in l})

{'taxonomy': [{'Kingdom': 'Plantae',
   'Clade': 'Asterids',
   'Order': 'Lamiales',
   'Family': 'Lamiaceae',
   'Genus': 'Monarda',
   'Species': 'M. fistulosa'}]}

## 
1. Put dictionary in database
    a. Use something like Unstructured database like MongoDB.
        * Each record is a dictionary.
        
3. 